# Matching amico with cosmoDC2_small

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import GCRCatalogs
import numpy as np
from astropy.table import Table, vstack
from astropy.io import fits
from astropy.io import ascii
import pandas as pd
###clevar
import clevar
from clevar import ClCatalog
from clevar.match import get_matched_pairs
from clevar.match import output_matched_catalog

### Listing variables names + read catalogs if needed

In [61]:
#print(cat.list_all_quantities())
#print(amico_data.colnames)
#cat = GCRCatalogs.load_catalog("cosmoDC2_v1.1.4_small")
"""
cosmodc2_data_mb = Table(cat.get_quantities(['ra', 'dec', 'mag_i', 'mag_r', 'mag_z', 'halo_id', 'halo_mass', 'position_x', 'position_y', 'redshift', 'galaxy_id', 'stellar_mass'], 
                                 filters=['halo_mass>1e14']))
cosmodc2_data_halo = Table(cat.get_quantities(['halo_id', 'ra', 'dec', 'redshift', 'halo_mass'], filters=['halo_mass > 1e14', 'is_central']))
cosmodc2_data_mb.write('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_mb_1e14.fits')
cosmodc2_data_halo.write('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_halo_1e14.fits')

cosmodc2_mb = Table.read('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_mb.fits')
cosmodc2_mb_inpath = '/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_mb.fits'
cosmodc2_halo = Table.read('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_halo.fits')
cosmodc2_halo_inpath = '/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_halo.fits'
"""

"\ncosmodc2_data_mb = Table(cat.get_quantities(['ra', 'dec', 'mag_i', 'mag_r', 'mag_z', 'halo_id', 'halo_mass', 'position_x', 'position_y', 'redshift', 'galaxy_id', 'stellar_mass'], \n                                 filters=['halo_mass>1e14']))\ncosmodc2_data_halo = Table(cat.get_quantities(['halo_id', 'ra', 'dec', 'redshift', 'halo_mass'], filters=['halo_mass > 1e14', 'is_central']))\ncosmodc2_data_mb.write('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_mb_1e14.fits')\ncosmodc2_data_halo.write('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_halo_1e14.fits')\n\ncosmodc2_mb = Table.read('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_mb.fits')\ncosmodc2_mb_inpath = '/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_mb.fits'\ncosmodc2_halo = Table.read('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_halo.fits')\ncosmodc2_halo_inpath = '/pbs/home/n/nam

------------

### Loading catalogs and converting them to clevar tables

In [62]:
cat_names = ['9943_map_associations', '9942_map_associations', '9816_map_associations', '9815_map_associations', '9814_map_associations', '9687_map_associations', '9686_map_associations',
             '9559_map_associations', '10450_map_associations', '10327_map_associations', '10326_map_associations', '10200_map_associations', '10199_map_associations',
             '10198_map_associations', '10072_map_associations', '10071_map_associations']
tile_names = []
for i in range(len(cat_names)):
    tile_names.append(cat_names[i].split("_")[0])

In [88]:
cosmodc2_mb_1e14 = Table.read('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_mb_1e14.fits')
cosmodc2_mb_inpath_1e14 = '/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_mb_1e14.fits'
cosmodc2_halo_1e14 = Table.read('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_halo_1e14.fits')
cosmodc2_halo_inpath_1e14 = '/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/dc2_cat/cdc2_halo_1e14.fits'

mb_inpath = '/sps/lsst/groups/clusters/amico_validation_project/catalogs/amico_cats/amico_map_associations/9943_map_associations.fits'
cl_inpath = '/sps/lsst/users/tguillem/DESC/desc_april_2022/cluster_challenge/clevar_catalogs/amico/map_detections_refined_noBuffer_all.fits'
amico_mb = Table.read(mb_inpath)
amico_cl = Table.read(cl_inpath)['ID', 'Xphys', 'Yphys', 'Zphys','LAMBSTAR', 'SN_NO_CLUSTER', 'SN', 'uid', 'tile']

In [134]:
amico_cl = amico_cl[(amico_cl['tile'] == 9943)]

In [125]:
c1  = ClCatalog.read(cosmodc2_halo_inpath_1e14, 'c1', tags={'id':'halo_id', 'mass':'halo_mass'}, full=True)

In [145]:
c2  = ClCatalog('c2', id = amico_cl['uid'], mass = amico_cl['LAMBSTAR'], ra = amico_cl['Xphys'], dec = amico_cl['Yphys'], redshift = amico_cl['Zphys'])

In [138]:
c1.read_members(cosmodc2_mb_inpath_1e14, tags={'id':'galaxy_id', 'id_cluster':'halo_id'}, full=True)

In [139]:
c2.read_members(mb_inpath, tags={'id':'mb_id', 'id_cluster':'uid'}, full=True)

In [141]:
from clevar.match import MembershipMatch
mt = MembershipMatch()

In [142]:
match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'preference': 'shared_member_fraction', # other options are more_massive, angular_proximity or redshift_proximity
    'minimum_share_fraction': 0,
    'match_members_kwargs': {'method':'id'},
}

In [143]:
mt.match_from_config(c1, c2, match_config)

3,490 members were matched.

## Multiple match (catalog 1)
Finding candidates (c1)
* 61/1,028 objects matched.es

## Multiple match (catalog 2)
Finding candidates (c2)
* 144/642 objects matched.

## Finding unique matches of catalog 1
Unique Matches (c1)
* 60/1,028 objects matched.

## Finding unique matches of catalog 2
Unique Matches (c2)
* 61/642 objects matched.
Cross Matches (c1)
* 60/1,028 objects matched.
Cross Matches (c2)
* 60/642 objects matched.
